In [1]:
import json
import numpy as np
import pandas as pd
import requests
import numpy
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

# Research Question: Find the top 10 players with the highest ratio between completed passes and attempted passes for La Liga

### Import the players dataset

In [3]:

players=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\players.json")
players
#Displaying maximum number
pd.set_option('display.max_columns', 14)
players.head()

,passportArea,weight,firstName,middleName,lastName,currentTeamId,birthDate,height,role,birthArea,wyId,foot,shortName,currentNationalTeamId
0,"{'name': 'Turkey', 'id': '792', 'alpha3code': ...",78,Harun,,Tekin,4502,1989-06-17,187,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'Turkey', 'id': '792', 'alpha3code': ...",32777,right,H. Tekin,4687
1,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",73,Malang,,Sarr,3775,1999-01-23,182,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393228,left,M. Sarr,4423
2,"{'name': 'France', 'id': '250', 'alpha3code': ...",72,Over,,Mandanda,3772,1998-10-26,176,"{'code2': 'GK', 'code3': 'GKP', 'name': 'Goalk...","{'name': 'France', 'id': '250', 'alpha3code': ...",393230,,O. Mandanda,null
3,"{'name': 'Senegal', 'id': '686', 'alpha3code':...",82,Alfred John Momar,,N'Diaye,683,1990-03-06,187,"{'code2': 'MD', 'code3': 'MID', 'name': 'Midfi...","{'name': 'France', 'id': '250', 'alpha3code': ...",32793,right,A. N'Diaye,19314
4,"{'name': 'France', 'id': '250', 'alpha3code': ...",84,Ibrahima,,Konat\u00e9,2975,1999-05-25,192,"{'code2': 'DF', 'code3': 'DEF', 'name': 'Defen...","{'name': 'France', 'id': '250', 'alpha3code': ...",393247,right,I. Konat\u00e9,null


### Import the event data for Spain

In [4]:
events_esp=pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\events\\events_Spain.json")
pd.set_option('display.max_columns', 15)

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422


#### As we have to calculate the efficiency of the players based on the passes, we filter out the information related to passes. Thus we target the column 'eventName' and choose only those rows which contains information related to passes. 

In [4]:
events_esp['eventName']=events_esp['eventName'].astype(str)
events_esp=events_esp[(events_esp.eventName.str.contains('Pass'))]
pd.set_option('display.max_columns', 15)
events_esp.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],3542,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548,Pass,682,1H,2.994582,85,180864419
1,8,Simple pass,[{'id': 1801}],274435,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548,Pass,682,1H,3.137020,85,180864418
2,8,Simple pass,[{'id': 1801}],364860,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548,Pass,682,1H,6.709668,85,180864420
3,8,Simple pass,[{'id': 1801}],3534,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548,Pass,682,1H,8.805497,85,180864421
4,8,Simple pass,[{'id': 1801}],3695,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548,Pass,682,1H,14.047492,85,180864422
...,...,...,...,...,...,...,...,...,...,...,...,...
628650,8,Hand pass,[{'id': 1801}],3486,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927,Pass,682,2H,2925.547631,81,253302662
628652,8,Simple pass,[{'id': 1801}],3695,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927,Pass,682,2H,2931.782904,85,253302665
628654,8,Simple pass,[{'id': 1801}],20623,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927,Pass,682,2H,2939.077491,85,253302671
628656,8,Cross,"[{'id': 401}, {'id': 1802}]",122832,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927,Pass,682,2H,2942.098761,80,253302674


#### Splitting the column 'Tags' that is a dictionary

In [5]:
TAG1=[]
for i in range(len(events_esp.tags)):
    try:
        TAG1.append(str(events_esp.tags[i][0]['id']))
    except:
        TAG1.append('Notag')
TAG2=[]
for i in range(len(events_esp.tags)):
    try:
        TAG2.append(str(events_esp.tags[i][1]['id']))
    except:
        TAG2.append('Notag')
TAG3=[]
for i in range(len(events_esp.tags)):
    try:
        TAG3.append(str(events_esp.tags[i][2]['id']))
    except:
        TAG3.append('Notag')
TAG4=[]
for i in range(len(events_esp.tags)):
    try:
        TAG4.append(str(events_esp.tags[i][2]['id']))
    except:
        TAG4.append('Notag')
TAG1=pd.DataFrame({'Tag1':TAG1})
TAG2=pd.DataFrame({'Tag2':TAG2})
TAG3=pd.DataFrame({'Tag3':TAG3})
TAG4=pd.DataFrame({'Tag4':TAG4})
TAG=[TAG1,TAG2,TAG3,TAG4]
TAG=pd.concat(TAG ,axis=1, sort=False)
mix_events_esp=[events_esp,TAG]
events_esp=pd.concat(mix_events_esp,  axis=1,  sort=False).reindex(events_esp.index)
events_esp=events_esp.drop('tags',axis=1)
events_esp

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,3542.0,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548.0,Pass,682.0,1H,2.994582,85,180864419.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,274435.0,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548.0,Pass,682.0,1H,3.137020,85,180864418.0,1801,Notag,Notag,Notag
2,8.0,Simple pass,364860.0,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548.0,Pass,682.0,1H,6.709668,85,180864420.0,1801,Notag,Notag,Notag
3,8.0,Simple pass,3534.0,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548.0,Pass,682.0,1H,8.805497,85,180864421.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,3695.0,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548.0,Pass,682.0,1H,14.047492,85,180864422.0,1801,Notag,Notag,Notag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628650,8.0,Hand pass,3486.0,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927.0,Pass,682.0,2H,2925.547631,81,253302662.0,NaN,NaN,NaN,NaN
628652,8.0,Simple pass,3695.0,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927.0,Pass,682.0,2H,2931.782904,85,253302665.0,NaN,NaN,NaN,NaN
628654,8.0,Simple pass,20623.0,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927.0,Pass,682.0,2H,2939.077491,85,253302671.0,NaN,NaN,NaN,NaN
628656,8.0,Cross,122832.0,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927.0,Pass,682.0,2H,2942.098761,80,253302674.0,NaN,NaN,NaN,NaN


In [6]:
events_esp['eventName'] = events_esp['eventName'].astype(str)
events_esp=events_esp[(events_esp.eventName.str.contains('Pass'))]
events_esp

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,3542.0,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548.0,Pass,682.0,1H,2.994582,85,180864419.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,274435.0,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548.0,Pass,682.0,1H,3.137020,85,180864418.0,1801,Notag,Notag,Notag
2,8.0,Simple pass,364860.0,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548.0,Pass,682.0,1H,6.709668,85,180864420.0,1801,Notag,Notag,Notag
3,8.0,Simple pass,3534.0,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548.0,Pass,682.0,1H,8.805497,85,180864421.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,3695.0,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548.0,Pass,682.0,1H,14.047492,85,180864422.0,1801,Notag,Notag,Notag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628650,8.0,Hand pass,3486.0,"[{'y': 29, 'x': 8}, {'y': 19, 'x': 14}]",2565927.0,Pass,682.0,2H,2925.547631,81,253302662.0,NaN,NaN,NaN,NaN
628652,8.0,Simple pass,3695.0,"[{'y': 34, 'x': 36}, {'y': 52, 'x': 45}]",2565927.0,Pass,682.0,2H,2931.782904,85,253302665.0,NaN,NaN,NaN,NaN
628654,8.0,Simple pass,20623.0,"[{'y': 25, 'x': 66}, {'y': 2, 'x': 88}]",2565927.0,Pass,682.0,2H,2939.077491,85,253302671.0,NaN,NaN,NaN,NaN
628656,8.0,Cross,122832.0,"[{'y': 21, 'x': 97}, {'y': 26, 'x': 92}]",2565927.0,Pass,682.0,2H,2942.098761,80,253302674.0,NaN,NaN,NaN,NaN


#### Counting the number of passes with respect to the player id

In [7]:
passes=events_esp
passes=passes.drop('positions',axis=1)
passes.playerId=list(passes.playerId)
PLAYERS=[]
for i in range(len(passes.playerId)):
    PLAYERS.append(passes.playerId.iloc[i])
passes_player_esp=pd.DataFrame(PLAYERS , columns=['PlayerId'])
passes_player_esp['Numberofpasses'] = passes_player_esp.groupby('PlayerId')['PlayerId'].transform('count')
passes_player_esp

,PlayerId,Numberofpasses
0,3542.0,1753
1,274435.0,346
2,364860.0,2338
3,3534.0,1353
4,3695.0,1390
...,...,...
318717,3486.0,37
318718,3695.0,1390
318719,20623.0,490
318720,122832.0,16


#### In order to set the threshold of the number of passes, we calculate the first quartile of the number of passes. Then we set the lower limit as (1st quartile*1.5). The reason behind this is, while dealing with a set of variables, the upper and lower boundary is being set as  (3rd quartile*1.5) and (1st quartile*1.5) respectively. Here we are concerned about the players who have played very limited number of matches or insufficient amount of hours. That is why we set the lower limit as the only threshold.

In [10]:
#calculating 1st quartile
#quartile_1st= passes_player_esp.Numberofpasses.quantile(0.25)
#quartile_3rd= passes_player_esp.Numberofpasses.quantile(0.75)
#quartile_1st=int(quartile_1st)
#quartile_3rd=int(quartile_3rd)
#IQR=quartile_3rd-quartile_1st
#calculating lower bound
Mean=passes_player_esp.Numberofpasses.mean()
std=passes_player_esp.Numberofpasses.std()
Mean=int(Mean)
std=int(std)
lower_bound=Mean-std
ID_passes_threshold_esp=[]
frequency_passes_threshold_esp=[]
frequency_passes=[]
ID_passes=[]
for i in range(len(passes_player_esp.Numberofpasses)):
    frequency_passes.append(float(passes_player_esp.Numberofpasses.iloc[i]))
    ID_passes.append(float(passes_player_esp.PlayerId.iloc[i]))
for i in range(len(frequency_passes)):
    if frequency_passes[i]>lower_bound:
        frequency_passes_threshold_esp.append(int(frequency_passes[i]))
        ID_passes_threshold_esp.append(int(ID_passes[i]))

In [11]:
passes_threshold_esp=pd.DataFrame(ID_passes_threshold_esp, columns=['PlayerId'])
passes_threshold_esp.insert(1, "Numberofpasses",frequency_passes_threshold_esp, True)
passes_threshold_esp.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
passes_threshold_esp

,PlayerId,Numberofpasses
0,3542,1753
1,364860,2338
2,3534,1353
3,3695,1390
4,3277,1333
...,...,...
149219,112366,542
151293,20803,542
151302,424624,604
152935,364850,559


In [16]:
events_esp_tag2=events_esp.loc[events_esp['Tag2'] == '1801']
events_esp_tag1=events_esp.loc[events_esp['Tag1']=='1801']
events_esp_tag3=events_esp.loc[events_esp['Tag3']=='1801']
events_esp_tag4=events_esp.loc[events_esp['Tag4']=='1801']
events_esp_tag=[events_esp_tag1,events_esp_tag2,events_esp_tag3,events_esp_tag4]
success_esp=pd.concat(events_esp_tag, axis=0)
success_esp.head()

,eventId,subEventName,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,Tag1,Tag2,Tag3,Tag4
0,8.0,Simple pass,3542.0,"[{'y': 61, 'x': 37}, {'y': 50, 'x': 50}]",2565548.0,Pass,682.0,1H,2.994582,85,180864419.0,1801,Notag,Notag,Notag
1,8.0,Simple pass,274435.0,"[{'y': 50, 'x': 50}, {'y': 30, 'x': 45}]",2565548.0,Pass,682.0,1H,3.137020,85,180864418.0,1801,Notag,Notag,Notag
2,8.0,Simple pass,364860.0,"[{'y': 30, 'x': 45}, {'y': 12, 'x': 38}]",2565548.0,Pass,682.0,1H,6.709668,85,180864420.0,1801,Notag,Notag,Notag
3,8.0,Simple pass,3534.0,"[{'y': 12, 'x': 38}, {'y': 69, 'x': 32}]",2565548.0,Pass,682.0,1H,8.805497,85,180864421.0,1801,Notag,Notag,Notag
4,8.0,Simple pass,3695.0,"[{'y': 69, 'x': 32}, {'y': 37, 'x': 31}]",2565548.0,Pass,682.0,1H,14.047492,85,180864422.0,1801,Notag,Notag,Notag
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317979,8.0,Cross,20436.0,"[{'y': 10, 'x': 100}, {'y': 45, 'x': 94}]",2565741.0,Pass,701.0,2H,317.351789,80,222947342.0,401,801,1801,1801
318074,8.0,Cross,4094.0,"[{'y': 87, 'x': 68}, {'y': 44, 'x': 84}]",2565741.0,Pass,701.0,2H,663.900472,80,222947457.0,402,801,1801,1801
318411,8.0,Cross,32601.0,"[{'y': 18, 'x': 94}, {'y': 47, 'x': 95}]",2565741.0,Pass,701.0,2H,1797.379926,80,222947874.0,301,401,1801,1801
318415,8.0,Cross,20436.0,"[{'y': 15, 'x': 70}, {'y': 87, 'x': 97}]",2565741.0,Pass,701.0,2H,1860.660279,80,222947883.0,401,801,1801,1801


In [24]:
S=[]
for i in range(len(success_esp.playerId)):
   S.append(success_esp.playerId.iloc[i])
success_esp_new=pd.DataFrame(S , columns=['PlayerId'])
success_esp_new['Numberofpasses'] = success_esp_new.groupby('PlayerId')['PlayerId'].transform('count')
success_esp_new.drop_duplicates(subset='PlayerId', keep='first', inplace=True)
success_esp_new

,PlayerId,Numberofpasses
0,3542.0,766
1,274435.0,149
2,364860.0,1064
3,3534.0,583
4,3695.0,606
...,...,...
117636,7864.0,49
119552,3779.0,4
121694,247150.0,20
123531,3604.0,2


In [25]:
final_passes_esp=pd.merge(success_esp_new, passes_threshold_esp, on='PlayerId')
final_passes_esp=final_passes_esp.rename(columns={'Numberofpasses_x':'AccuratePassesNumber','Numberofpasses_y':'TotalPassesNumber'})
final_passes_esp

,PlayerId,AccuratePassesNumber,TotalPassesNumber
0,3542.0,766,1753
1,364860.0,1064,2338
2,3534.0,583,1353
3,3695.0,606,1390
4,3277.0,689,1333
...,...,...,...
249,7860.0,339,604
250,489488.0,137,659
251,7857.0,142,670
252,3866.0,76,573


In [26]:
def make_col(df, col):
    return pd.concat([df.drop([col], axis=1), df[col].apply(pd.Series)], axis=1)
teams = pd.read_json("C:\\Users\\Debodeep\\Documents\\Sapienza Learning Materials\\ADM\\ADMHw2\\teams.json").rename(columns={'name':'teamname'})
teams = make_col(teams, "area")
teams = teams.drop(columns='city')
teams = teams.rename(columns={'name':'nation'})
espteams = teams[(teams["alpha3code"]=="ESP") & (teams["type"]=="club")]
espplayer = players.merge(espteams, left_on="currentTeamId", right_on="wyId")
espplayer=espplayer.drop(columns=['birthArea','height','role','currentNationalTeamId','officialName','type','middleName','lastName','passportArea','weight','currentTeamId','firstName','foot'],axis=1)

espplayer=espplayer.drop(columns=['alpha3code','alpha2code','id','nation','wyId_y','teamname'])
espplayer=espplayer.rename(columns={"wyId_x": "PlayerId"})

final_player_passes_esp=pd.merge(espplayer, final_passes_esp, on='PlayerId')
final_player_passes_esp

,birthDate,PlayerId,shortName,AccuratePassesNumber,TotalPassesNumber
0,1988-11-20,254,R. Rosales,592,1475
1,1991-01-11,3568,Recio,682,887
2,1984-09-14,3645,Chory Castro,345,873
3,1988-05-25,3705,Adri\u00e1n,479,980
4,1986-02-10,3719,Roberto,241,508
...,...,...,...,...,...
229,1992-01-24,140046,M. Dmitrovi\u0107,303,731
230,1993-03-27,212644,G. Escalante,208,626
231,1994-07-06,247207,Joan Jord\u00e1n,546,1131
232,1989-09-05,20436,Jos\u00e9 \u00c1ngel,447,1202


In [27]:
Efficiency_esp=[]
for i in range(len(final_player_passes_esp.AccuratePassesNumber)):
    total=final_player_passes_esp.TotalPassesNumber.iloc[i]
    accurate=final_player_passes_esp.AccuratePassesNumber.iloc[i]
    Ratio=accurate/total
    Efficiency_esp.append(Ratio)
Efficiency_esp=pd.DataFrame(Efficiency_esp)

In [28]:
final_player_passes_esp.insert(4,'Efficiency',Efficiency_esp)
final_player_ratio_esp=final_player_passes_esp
final_player_ratio_esp

,birthDate,PlayerId,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1988-11-20,254,R. Rosales,592,0.401356,1475
1,1991-01-11,3568,Recio,682,0.768884,887
2,1984-09-14,3645,Chory Castro,345,0.395189,873
3,1988-05-25,3705,Adri\u00e1n,479,0.488776,980
4,1986-02-10,3719,Roberto,241,0.474409,508
...,...,...,...,...,...,...
229,1992-01-24,140046,M. Dmitrovi\u0107,303,0.414501,731
230,1993-03-27,212644,G. Escalante,208,0.332268,626
231,1994-07-06,247207,Joan Jord\u00e1n,546,0.482759,1131
232,1989-09-05,20436,Jos\u00e9 \u00c1ngel,447,0.371880,1202


In [30]:
final_player_ratio_esp.to_json(r'C:\Users\Debodeep\Documents\Sapienza Learning Materials\ADM\ADMHw2\final_player_ratio_esp.json')

In [31]:
bestefficiency_esp=final_player_ratio_esp.nlargest(10, ['Efficiency']) 
bestefficiency_esp=bestefficiency_esp.reset_index()
bestefficiency_esp=bestefficiency_esp.drop(columns=['index','PlayerId'])
bestefficiency_esp=bestefficiency_esp.set_value(9, 'shortName', 'V.Lindelof')

bestefficiency_esp

C:\Users\Debodeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


,birthDate,shortName,AccuratePassesNumber,Efficiency,TotalPassesNumber
0,1990-09-15,Bigas,504,0.904847,557
1,1989-11-14,Font\u00e0s,587,0.845821,694
2,1991-01-11,Recio,682,0.768884,887
3,1989-12-23,Z. Feddal,548,0.718218,763
4,1989-11-26,Juan Cala,390,0.707804,551
5,1993-01-20,Fede Cartabia,370,0.696798,531
6,1985-10-14,G. Cabral,825,0.676784,1219
7,1989-03-26,S. Kj\u00e6r,509,0.673280,756
8,1998-07-22,F. Valverde,363,0.656420,553
9,1992-03-27,V.Lindelof,382,0.648557,589


### The above table shows the top 10 most efficient players. We can observe that the highest efficiency is 90.48% in the premier league dataset.